In [ ]:
!pip install pyaudio
!pip install wave
!pip install numpy
!pip install faster-whisper
!pip install keyboard

In [ ]:
import pyaudio
import wave
import os
import numpy as np
from faster_whisper import WhisperModel
import keyboard
import threading

In [ ]:
# Global variables
is_recording = False
frames = []

In [ ]:
# Load the Faster Whisper model
model_size = "base"

# Run on GPU with FP16
model = WhisperModel(model_size, device="cuda", compute_type="float16")

In [ ]:
def record_audio(filename, sample_rate=16000):
    """Record audio from the microphone and save it to a file."""
    global is_recording, frames
    
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=sample_rate,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("Press SPACE to start recording...")
    keyboard.wait('space')
    print("Recording... Press SPACE again to stop.")
    
    is_recording = True
    frames = []

    while is_recording:
        data = stream.read(CHUNK)
        frames.append(data)

    print("Recording finished.")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    wf.close()

In [ ]:
def transcribe_audio(filename):
    """Transcribe the audio file using Faster Whisper."""
    segments, info = model.transcribe(filename, beam_size=5)
    return " ".join([segment.text for segment in segments])

In [ ]:
def on_press_space():
    global is_recording
    if is_recording:
        is_recording = False
        
# Set up keyboard listener
keyboard.on_press_key('space', lambda _: on_press_space())

In [ ]:
# Record audio
audio_filename = "audio.wav"
record_thread = threading.Thread(target=record_audio, args=(audio_filename,))
record_thread.start()
record_thread.join()

In [ ]:
# Transcribe the recorded audio
transcription = transcribe_audio(audio_filename)
print("Transcription:", transcription)

In [ ]:
# Clean up
keyboard.unhook_all()

In [ ]:
# Check if the output file exists and delete it if it does
if os.path.exists(audio_filename):
    os.remove(audio_filename)